In [ ]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2021-03-21 19:31:03--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  38.5MB/s    in 7.6s    

2021-03-21 19:31:11 (22.8 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2021-03-21 19:31:11--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  16.8MB/s    in 4.7s    

2021-03-21 19:31:16 (13.2 MB/s) - ‘test_32x32.mat’ saved [64275384/64

In [ ]:
import keras
from keras.layers.normalization import BatchNormalization
import numpy as np
import scipy.io as io
from keras.layers import Activation, Flatten,Softmax
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.models import Sequential
from keras import optimizers
from keras.backend import function
import pandas as pd
from keras.backend import function

In [ ]:
#importing data
train_set = io.loadmat("train_32x32.mat")
test_set = io.loadmat("test_32x32.mat")
x_train = train_set['X']
y_train = train_set['y']
x_test = test_set['X']
y_test = test_set['y']
# Reshaping test and training data sets
x_train = np.moveaxis(x_train,[3,0,1,2],[0,1,2,3])
x_train = x_train.astype('float32')
x_test = np.moveaxis(x_test,[3,0,1,2],[0,1,2,3])
x_test = np.asarray(x_test)
x_test = x_test.astype('float32')
#Normalisation des données
x_train /= x_train.max()
x_test /= x_test.max()
# Passage de classe comme integer à classe comme vecteur de dimension classes
y_test_RN = [y-1 for y in y_test]# la classe 1 corresponds au array qui à 1 en indice 0
y_train_RN = [y-1 for y in y_train]
classes = 10
y_train_RN = keras.utils.to_categorical(y_train_RN, classes)
y_test_RN = keras.utils.to_categorical(y_test_RN, classes)
 #Définition du type de data pour optimiser la mémoire
y_train_RN = y_train_RN.astype('int32')
y_test_RN = y_test_RN.astype('int32')
# Featurewise normalization
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=False,
    zca_whitening=False, 
    zca_epsilon=1e-06)

datagen.fit(x_train)

In [ ]:
y_test_KNN=[k.item() for k in y_test]
y_train_KNN=[k.item() for k in y_train]

In [ ]:
#from google.colab import drive
#drive.mount('drive')

In [ ]:
#model = keras.models.load_model('/content/drive/MyDrive/Projet 3A')

In [ ]:
def deepnet2():
  model=Sequential()
  model.add(Conv2D(64, kernel_size=(3, 3),padding = 'same', activation='relu', input_shape=x_train.shape[1:]))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),  padding = 'valid', name='S1'))
  model.add(Conv2D(128, kernel_size=(3, 3), padding = 'valid', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),  padding = 'same', name='S2'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Dropout(0.5)) 
  model.add(Flatten())
  model.add(Dense(classes, activation='softmax'))
  adam = optimizers.Adam(lr = 0.001)
  model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer= adam,
                  metrics=['accuracy'])
  return model

In [ ]:
model=deepnet2()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
S1 (MaxPooling2D)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
S2 (MaxPooling2D)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 128)         147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)         147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 1, 128)         1

In [ ]:
model.fit(x_train,y_train_RN, validation_split=0.2 ,epochs=10, batch_size=128)

Epoch 1/10
458/458 [==============================] - 269s 585ms/step - loss: 1.9465 - accuracy: 0.3069 - val_loss: 0.5897 - val_accuracy: 0.8238
Epoch 2/10
458/458 [==============================] - 271s 593ms/step - loss: 0.6199 - accuracy: 0.8163 - val_loss: 0.4292 - val_accuracy: 0.8696
Epoch 3/10
458/458 [==============================] - 276s 603ms/step - loss: 0.4665 - accuracy: 0.8653 - val_loss: 0.3811 - val_accuracy: 0.8904
Epoch 4/10
458/458 [==============================] - 278s 608ms/step - loss: 0.3847 - accuracy: 0.8897 - val_loss: 0.3401 - val_accuracy: 0.9006
Epoch 5/10
458/458 [==============================] - 285s 622ms/step - loss: 0.3364 - accuracy: 0.9046 - val_loss: 0.3125 - val_accuracy: 0.9086
Epoch 6/10
458/458 [==============================] - 281s 613ms/step - loss: 0.3041 - accuracy: 0.9131 - val_loss: 0.3101 - val_accuracy: 0.9127
Epoch 7/10
458/458 [==============================] - 280s 611ms/step - loss: 0.2818 - accuracy: 0.9212 - val_loss: 0.2787 -

In [ ]:
model.evaluate(x_test[:5000],y_test_RN[:5000])

157/157 [==============================] - 5s 34ms/step - loss: 0.3206 - accuracy: 0.9208


[0.3206191956996918, 0.920799970626831]

In [ ]:
scores=[]
x_train_10000=x_train[:15000]

In [ ]:
def generalpooling(X):
  (l,m,n)=X.T.shape
  L=[]
  X1=X.reshape(l,m*n)
  for k in range(l):
    L.append(max(X1[k]))
  return L




In [ ]:
def generalpooling_data(d):
  D=[]
  for X in d:
    D.append(generalpooling(X))
  return D


In [ ]:
m=5

#Conv2D 50

In [ ]:
get_conv50_output = function([model.layers[0].input],
                                  [model.layers[6].output])
layer_output_conv50 = get_conv50_output([x_train_10000])

In [ ]:
layer_output_conv50[0][0].shape

(1, 1, 128)

In [ ]:
conv50=np.array(layer_output_conv50[0]).reshape(15000,128)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_conv50=KNeighborsClassifier(m)
knn_conv50.fit(conv50,y_train_KNN[:15000])
layer_output_conv50_test = get_conv50_output([x_test[:5000]])
conv50_test=np.array(layer_output_conv50_test[0]).reshape(5000,128)
score=knn_conv50.score(conv50_test,y_test_KNN[:5000])
scores.append(score)

Pas besoin de faire le generalpooling sur cette couche car les cartes sont composé d'une seule valeur


In [ ]:
#scores.append(0.9142)

In [ ]:
scores

[0.9178]

#Conv2D 40


In [ ]:
get_conv40_output = function([model.layers[0].input],
                                  [model.layers[5].output])
layer_output_conv40 = get_conv40_output([x_train_10000])[0]

In [ ]:
layer_output_conv40_test = get_conv40_output([x_test[:5000]])[0]

In [ ]:
conv40=generalpooling_data(layer_output_conv40)

In [ ]:
conv40_test=generalpooling_data(layer_output_conv40_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_conv40=KNeighborsClassifier(m)
knn_conv40.fit(conv40,y_train_KNN[:15000])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
score=knn_conv40.score(conv40_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)

In [ ]:
scores

[0.9178, 0.8766]

# Conv2D 30


In [ ]:
get_conv30_output = function([model.layers[0].input],
                                  [model.layers[4].output])
layer_output_conv30 = get_conv30_output([x_train_10000])[0]

In [ ]:
layer_output_conv30_test = get_conv30_output([x_test[:5000]])[0]

In [ ]:
conv30=generalpooling_data(layer_output_conv30)

In [ ]:
conv30_test=generalpooling_data(layer_output_conv30_test)

In [ ]:
layer_output_conv30.shape

In [ ]:
np.array(conv30).shape

In [ ]:
knn_conv30=KNeighborsClassifier(m)
knn_conv30.fit(conv30,y_train_KNN[:10000])

In [ ]:
score=knn_conv30.score(conv30_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)
scores

#Maxpooling S2

In [ ]:
get_S2_output = function([model.layers[0].input],
                                  [model.layers[3].output])
layer_output_S2 = get_S2_output([x_train_10000])[0]

In [ ]:
layer_output_S2_test = get_S2_output([x_test[:5000]])[0]

In [ ]:
S2=generalpooling_data(layer_output_S2)

In [ ]:
S2_test=generalpooling_data(layer_output_S2_test)

In [ ]:
knn_S2=KNeighborsClassifier(m)
knn_S2.fit(S2,y_train_KNN[:10000])

In [ ]:
score=knn_S2.score(S2_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)
scores

# Conv2D 20

In [ ]:
get_conv20_output = function([model.layers[0].input],
                                  [model.layers[2].output])
layer_output_conv20 = get_conv20_output([x_train_10000])[0]

In [ ]:
layer_output_conv20_test = get_conv20_output([x_test[:5000]])[0]

In [ ]:
conv20=generalpooling_data(layer_output_conv20)

In [ ]:
conv20_test=generalpooling_data(layer_output_conv20_test)

In [ ]:
knn_conv20=KNeighborsClassifier(m)
knn_conv20.fit(conv20,y_train_KNN[:10000])

In [ ]:
score=knn_conv20.score(conv20_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)
scores